# Log Semiring

[Log semirings](https://en.wikipedia.org/wiki/Log_semiring) are useful for networks of [log probabilities](https://en.wikipedia.org/wiki/Log_probability).  As small probabilities multiply, they get even smaller, pushing the limits of floating point accuracy.  In a log semiring, the value is exponentiated first, providing a large positive (or zero) number.  "Multiplication" of two exponetiated operands is then just arithmetic addition which is typically faster than multiplying.  When converted back to Python floats, the exponentiated to reverse the operation.

In [1]:
from pygraphblas import *
from math import log, log1p, exp

class Log32(Scalar, FP32):
    
    in_func = log
    out_func = exp  
    
    @binop()
    def add_op(z, x, y):
        z = x + log1p(exp(y - x))
    
    @binop()
    def mult_op(z, x, y):
        z = x + y


RuntimeError: Error calling __set_name__ on 'inner' instance 'add_op' in 'Log32'

In [ ]:
A = Matrix.from_type(Log32, 6, 6)
A[0,1] = 1/9.0
A[0,3] = 1/3.0
A[1,2] = 1/8.0
A[3,4] = 1/6.0
A[3,5] = 1/1.0
A[4,2] = 1/4.0
A[1,5] = 1/7.0
A[5,4] = 1/2.0
from pygraphblas.demo.gviz import draw
draw(A, label_width=5)

In [ ]:
draw(A @ A, label_width=5)